<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/GO_referral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')

!pip install japanize_matplotlib --q

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# フォント設定を最初に行う
plt.rcParams['font.family'] = 'IPAexGothic'
sns.set(font='IPAexGothic', font_scale=2.5)  # font_scale を2.5に増加
plt.rcParams['axes.unicode_minus'] = False

def load_and_prepare_data(file_path):
    """データの読み込みと前処理を行う関数"""
    df = pd.read_excel(file_path, sheet_name='A0403_検査結果')

    referral_code_map = {
        "0": "近隣の内科",
        "1": "院内の内科",
        "2": "近隣の眼科",
        "3": "基幹病院眼科",
    }

    df['紹介元'] = df[
        "0近隣内科、1院内内科、2近隣眼科、3基幹病院眼科、4当院眼科、5.その他"
    ].astype(str).map(referral_code_map)

    df['いつから（M）'] = pd.to_numeric(df['いつから（M）'], errors='coerce')

    return df

def filter_steroid_cases(df):
    """ステロイド治療症例のフィルタリング"""
    steroid_columns = [col for col in df.columns if "ステロイド" in col]
    if not steroid_columns:
        print("警告: ステロイド治療の列が見つかりませんでした。全データを使用します。")
        return df

    return df[df[steroid_columns[0]] != 0]

def create_modern_boxplot(df, categories):
    """モダンな箱ひげ図の作成（seabornを使用）"""
    # プロット設定
    plt.figure(figsize=(16, 14))

    df['紹介元'] = pd.Categorical(df['紹介元'], categories=categories)

    # 箱ひげ図
    ax = sns.boxplot(
        data=df,
        x='紹介元',
        y='いつから（M）',
        order=categories,
        palette='Set3',
        showfliers=True,
        medianprops={"color": "red", "linewidth": 3},
        boxprops={"alpha": 0.7, "linewidth": 2},
        flierprops={'marker': 'o', 'markerfacecolor': 'gray', 'alpha': 0.3, 'markersize': 12}
    )

    # スウォームプロット
    sns.swarmplot(
        data=df,
        x='紹介元',
        y='いつから（M）',
        order=categories,
        color='0.25',
        size=8,
        alpha=0.7,
        dodge=True
    )

    # グラフの設定
    plt.xlabel('')
    plt.ylabel('経過月数', fontsize=32, fontweight='bold')  # さらに大きく
    plt.title('紹介元別の経過月数分布\n（ステロイド治療症例）',
              fontsize=36, pad=20, fontweight='bold')  # さらに大きく

    plt.ylim(-2, 15)

    # グリッド線
    plt.grid(axis='y', linestyle='--', alpha=0.3, linewidth=2)

    # 軸の設定
    plt.xticks(rotation=0, fontsize=28)  # さらに大きく
    plt.yticks(fontsize=28)  # さらに大きく

    # 余白の調整
    plt.subplots_adjust(bottom=0.2, left=0.15)  # 左余白を増やして軸ラベルが切れないように

    # 統計情報
    for i, category in enumerate(categories):
        data = df[df['紹介元'] == category]['いつから（M）'].dropna()
        if len(data) > 0:
            median = data.median()
            count = len(data)
            plt.text(i, -1.5, f'症例数={count}\n中央値={median:.1f}ヶ月',
                    horizontalalignment='center',
                    size=24,  # さらに大きく
                    color='dimgray',
                    fontweight='bold')

    # タイトルと軸ラベルが切れないように調整
    plt.tight_layout()
    return plt.gcf()

def main():
    file_path = "/content/drive/MyDrive/発表/アムジェン社内講演20250214/甲状腺眼症ステロイド紹介元.xlsx"

    display_order = [
        "近隣の内科",
        "院内の内科",
        "近隣の眼科",
        "基幹病院眼科",
    ]

    df = load_and_prepare_data(file_path)
    df_filtered = filter_steroid_cases(df)
    fig = create_modern_boxplot(df_filtered, display_order)
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# フォント設定
plt.rcParams['font.family'] = 'IPAexGothic'
sns.set(font='IPAexGothic', font_scale=1.5)
plt.rcParams['axes.unicode_minus'] = False

# データの読み込み
file_path = "/content/drive/MyDrive/発表/アムジェン社内講演20250214/甲状腺眼症ステロイド紹介元.xlsx"
df = pd.read_excel(file_path, sheet_name='A0403_検査結果')

# クロス集計の作成
cross_tab = pd.crosstab(
    df['0近隣内科、1院内内科、2近隣眼科、3基幹病院眼科、4当院眼科、5.その他'],
    df['0バセドウ、1橋本病、2Euthyroid']
)

# インデックス（紹介元）の日本語化
cross_tab.index = cross_tab.index.map({
    0: '近隣内科',
    1: '院内内科',
    2: '近隣眼科',
    3: '基幹病院眼科',
    4: '当院眼科',
    5: 'その他'
})

# 列（疾患）の日本語化
cross_tab.columns = ['バセドウ', '橋本病', 'Euthyroid']

# 全体の集計を追加
total_row = pd.DataFrame(cross_tab.sum()).T
total_row.index = ['全体']

# 全体を一番上に追加
cross_tab_with_total = pd.concat([total_row, cross_tab])

# 100%積み上げ棒グラフ用データの作成（割合に変換）
percentage_df = cross_tab_with_total.div(cross_tab_with_total.sum(axis=1), axis=0) * 100

# グラフ作成
fig, ax = plt.subplots(figsize=(12, 7))
percentage_df.plot(kind='bar', stacked=True, ax=ax)

# タイトルとラベルの設定
ax.set_title("紹介元別疾患分布（100%積み上げ棒グラフ）", fontsize=20)
ax.set_xlabel("紹介元", fontsize=16)
ax.set_ylabel("割合 (%)", fontsize=16)

# 凡例の設定
ax.legend(title="疾患", fontsize=16, title_fontsize=16, loc='center left', bbox_to_anchor=(1.0, 0.5))

# 全体（一番左の棒）にのみ実数を表示
y_offset = 0
for j in range(len(cross_tab_with_total.columns)):
    value = cross_tab_with_total.iloc[0, j]  # 最初の行（全体）のみ
    percentage = percentage_df.iloc[0, j]
    if percentage > 0:  # 0%より大きい場合のみ表示
        ax.text(0, y_offset + percentage/2,
               f'{value:,.0f}\n({percentage:.1f}%)',
               ha='center', va='center')
    y_offset += percentage

# レイアウトの調整
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 紹介元の内訳データを集計
referral_distribution = df['0近隣内科、1院内内科、2近隣眼科、3基幹病院眼科、4当院眼科、5.その他'].value_counts().sort_index()

# 紹介元名を日本語にマッピング
referral_distribution.index = referral_distribution.index.map({
    0: '近隣内科',
    1: '院内内科',
    2: '近隣眼科',
    3: '基幹病院眼科',
    4: '当院眼科',
    5: 'その他'
})

# パーセンテージと絶対数を表示する関数
def autopct_format(pct, all_values):
    absolute = int(round(pct/100 * sum(all_values)))
    return f'{pct:.1f}%\n({absolute})'  # 改行でパーセンテージと絶対数を分けて表示

# 円グラフの作成
plt.figure(figsize=(12, 12))
colors = sns.color_palette("pastel", len(referral_distribution))
wedges, texts, autotexts = plt.pie(
    referral_distribution,
    labels=referral_distribution.index,
    autopct=lambda pct: autopct_format(pct, referral_distribution),
    colors=colors,
    startangle=140,
    wedgeprops={'edgecolor': 'white', 'linewidth': 2},
    textprops={'fontsize': 28},  # ラベル文字サイズ
    pctdistance=0.8  # パーセンテージ位置を調整
)

# 自動テキスト（パーセンテージ・絶対数）のフォントサイズ調整
for autotext in autotexts:
    autotext.set_fontsize(28)
    autotext.set_weight('bold')  # 太字で見やすく

# タイトルの設定
plt.title("紹介元の内訳 (n=120)", fontsize=30, weight='bold')
plt.axis('equal')  # 円を正円に保つ
plt.show()


In [ ]:
# "主訴"列の値を"、"で分割し、個別の症状としてカウントする
chief_complaints_expanded = df['主訴'].dropna().str.split('、').explode()

# 症状ごとの出現頻度をカウント
chief_complaints_split_stats = chief_complaints_expanded.value_counts().reset_index()
chief_complaints_split_stats.columns = ['症状', '件数']

# 症状の統合ルールを辞書で定義
symptom_mapping = {
    '眼瞼後退': '眼瞼後退',
    '上眼瞼後退': '眼瞼後退',
    '眼球運動制限': '眼瞼後退',
    '眼痛': '眼痛',
    '球後痛': '眼痛',
    '眼球運動痛': '眼痛',
    '痛み': '眼痛',
    '球後の違和感': '眼痛',
    '視神経症': '視神経症',
    '視力低下': '視神経症',
    '斜視': '複視'
}

# 症状列をマッピングして統合
chief_complaints_split_stats['統合症状'] = chief_complaints_split_stats['症状'].map(symptom_mapping).fillna(chief_complaints_split_stats['症状'])

# 統合後の症状ごとの件数を集計
consolidated_symptoms_stats = chief_complaints_split_stats.groupby('統合症状')['件数'].sum().reset_index().sort_values(by='件数', ascending=False)

# 結果を表示
consolidated_symptoms_stats


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

# フォント設定
plt.rcParams['font.family'] = 'IPAexGothic'
sns.set(font='IPAexGothic', style='whitegrid')

# データの読み込み
# このパスは、お使いの環境に合わせて適宜修正してください。
file_path = "/content/drive/MyDrive/発表/アムジェン社内講演20250214/甲状腺眼症ステロイド紹介元.xlsx"
df = pd.read_excel(file_path, sheet_name='A0403_検査結果')

# 症状を処理・統合する関数
def process_symptoms(df, symptom_mapping):
    """主訴を処理し、統合する関数"""
    if '主訴' not in df.columns:
        return pd.DataFrame(columns=['統合症状', '件数'])

    chief_complaints_expanded = df['主訴'].dropna().str.split('、').explode()
    if chief_complaints_expanded.empty:
        return pd.DataFrame(columns=['統合症状', '件数'])

    chief_complaints_stats = chief_complaints_expanded.value_counts().reset_index()
    chief_complaints_stats.columns = ['症状', '件数']

    chief_complaints_stats['統合症状'] = chief_complaints_stats['症状'].map(symptom_mapping).fillna(chief_complaints_stats['症状'])
    consolidated_stats = chief_complaints_stats.groupby('統合症状')['件数'].sum().reset_index()

    return consolidated_stats.sort_values(by='件数', ascending=False)

# 症状の統合ルール
symptom_mapping = {
    '眼瞼後退': '眼瞼後退', '上眼瞼後退': '眼瞼後退', '眼球運動制限': '眼瞼後退',
    '眼痛': '眼痛', '球後痛': '眼痛', '眼球運動痛': '眼痛', '痛み': '眼痛', '球後の違和感': '眼痛',
    '視神経症': '視神経症', '視力低下': '視神経症',
    '斜視': '複視'
}

# 紹介元の列を特定
referral_col = "0近隣内科、1院内内科、2近隣眼科、3基幹病院眼科、4当院眼科、5.その他"

# 紹介元をカテゴリ分類
df['紹介元カテゴリ'] = df[referral_col].astype(str).map({
    "0": "内科", "1": "内科", "2": "眼科", "3": "眼科",
    "4": "その他", "5": "その他"
})

# 内科と眼科のデータフレームを抽出
df_naika = df[df['紹介元カテゴリ'] == '内科']
df_ganka = df[df['紹介元カテゴリ'] == '眼科']

# それぞれのグループで症状を集計
symptoms_naika = process_symptoms(df_naika, symptom_mapping)
symptoms_ganka = process_symptoms(df_ganka, symptom_mapping)

# 結果をマージして比較用のデータフレームを作成
comparison_df = pd.merge(symptoms_naika, symptoms_ganka, on='統合症状', how='outer', suffixes=('_内科', '_眼科')).fillna(0)
comparison_df['件数_内科'] = comparison_df['件数_内科'].astype(int)
comparison_df['件数_眼科'] = comparison_df['件数_眼科'].astype(int)
comparison_df = comparison_df.set_index('統合症状')
comparison_df['合計'] = comparison_df['件数_内科'] + comparison_df['件数_眼科']
comparison_df = comparison_df.sort_values(by='合計', ascending=False).drop(columns='合計')

# グラフの描画
ax = comparison_df.plot(kind='bar', figsize=(18, 10), width=0.8, colormap='Paired')

plt.title('内科紹介と眼科紹介における主訴の分布比較', fontsize=24, fontweight='bold', pad=20)
plt.ylabel('件数', fontsize=18, fontweight='bold')
plt.xlabel('症状', fontsize=18, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=16)
plt.yticks(fontsize=16)
plt.legend(['内科からの紹介', '眼科からの紹介'], fontsize=16, title='紹介元')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 棒グラフの上に件数を表示
for p in ax.patches:
    if p.get_height() > 0:
        ax.annotate(f'{int(p.get_height())}',
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center',
                    xytext=(0, 9),
                    textcoords='offset points',
                    fontsize=12,
                    fontweight='bold')

plt.tight_layout()
plt.show()

# 集計表の表示
print("\n内科紹介と眼科紹介の主訴（統合後）のクロス集計:")
print(comparison_df)

In [ ]:
# 「手術」という列を確認して、その中で特定の文字列が何回出現するかを数える

# 該当する列名を確認
columns = df.columns

# 「手術」に該当する列を検索
surgery_col = [col for col in columns if '手術' in col]

# 手術列が存在するか確認
if surgery_col:
    surgery_column = surgery_col[0]
    # 各キーワードの出現回数をカウント
    count_ganka = df[surgery_column].astype(str).str.contains("眼窩", na=False).sum()
    count_shashi = df[surgery_column].astype(str).str.contains("斜視", na=False).sum()
    count_ganken = df[surgery_column].astype(str).str.contains("眼瞼", na=False).sum()
else:
    count_ganka = count_shashi = count_ganken = 0

count_ganka, count_shashi, count_ganken


(np.int64(19), np.int64(15), np.int64(7))

In [ ]:
# 紹介元と甲状腺疾患診断済みの列名を設定
referral_col = "0近隣内科、1院内内科、2近隣眼科、3基幹病院眼科、4当院眼科、5.その他"
thyroid_diagnosis_col = [col for col in df.columns if '甲状腺疾患診断済み' in col][0]

# 「紹介元」が「2近隣眼科」または「3基幹病院眼科」の患者を抽出
referral_filtered_df = df[df[referral_col].astype(str).isin(['2', '3'])]

# 甲状腺疾患診断済み（M）列で数字が書いてあるものをカウント
count_numeric = referral_filtered_df[thyroid_diagnosis_col].astype(str).str.match(r'^\d+$').sum()

# 「未」となっているものをカウント
count_undetermined = referral_filtered_df[thyroid_diagnosis_col].astype(str).str.strip().eq('未').sum()

# グラフ用のデータ
labels = ["甲状腺機能異常診断済み（非連携内科に通院中）", "甲状腺機能異常未診断例"]
counts = [count_numeric, count_undetermined]

# 円グラフの作成（割合と数を表示）
def make_autopct(values):
    def autopct(pct):
        total = sum(values)
        count = int(round(pct * total / 100.0))
        return f"{pct:.1f}%\n({count}件)"
    return autopct

import seaborn as sns

# Seabornのテーマを設定
sns.set_theme(style="whitegrid")
sns.set(font='IPAexGothic', font_scale=2.5)  # font_scale を2.5に増加


# 色パレットの設定
colors = sns.color_palette("pastel")

# 円グラフの作成（Seabornスタイルでモダンに表示）
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=labels, colors=colors, autopct=make_autopct(counts), startangle=90, counterclock=False, wedgeprops={'edgecolor': 'white'})
plt.title("近隣眼科・基幹病院眼科からの紹介患者における甲状腺機能異常の診断状況", fontsize=14, weight='bold')
plt.tight_layout()

# グラフの表示
plt.show()

In [ ]:
# 「0バセドウ、1橋本病、2Euthyroid」列を取得
thyroid_type_col = [col for col in df.columns if '0バセドウ' in col or '1橋本病' in col or '2Euthyroid' in col][0]

# 既存の条件を満たすデータフレームの中からEuthyroidの患者数をカウント
count_euthyroid = referral_filtered_df[thyroid_type_col].astype(str).str.strip() == '2'
count_euthyroid = count_euthyroid.sum()

# グラフ用のデータを更新
labels = [
    "甲状腺機能異常診断済み（非連携内科に通院中）",
    "甲状腺機能異常未診断例",
    "Euthyroid群"
]
counts = [count_numeric, count_undetermined, count_euthyroid]

# Seabornのカラーパレットを設定
colors = sns.color_palette("pastel", len(labels))

# 円グラフの作成（割合と数を表示）
plt.figure(figsize=(8, 8))
plt.pie(
    counts,
    labels=labels,
    colors=colors,
    autopct=make_autopct(counts),
    startangle=90,
    counterclock=False,
    wedgeprops={'edgecolor': 'white'}
)
plt.title("近隣眼科・基幹病院眼科からの紹介患者における甲状腺機能異常の診断状況", fontsize=14, weight='bold')
plt.tight_layout()

# グラフの表示
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, mannwhitneyu
import pandas as pd

# 必要な列の取得
steroid_col = [col for col in df.columns if 'ステロイド' in col][0]
surgery_col = [col for col in df.columns if '手術' in col][0]
onset_col = [col for col in df.columns if 'いつから' in col][0]

# ステロイド治療が1の群を抽出
steroid_group = df[df[steroid_col] == 1].copy()

# 手術群のラベル付けと数値変換
steroid_group['手術群'] = steroid_group[surgery_col].apply(lambda x: '手術なし' if pd.isna(x) or str(x).strip() == '' else '手術あり')
steroid_group['いつから（M）_数値'] = pd.to_numeric(steroid_group[onset_col], errors='coerce')

# 群ごとのデータ抽出
surgery_nil_group = steroid_group[steroid_group['手術群'] == '手術なし']['いつから（M）_数値'].dropna()
surgery_not_nil_group = steroid_group[steroid_group['手術群'] == '手術あり']['いつから（M）_数値'].dropna()

# 正規性検定
shapiro_nil = shapiro(surgery_nil_group)
shapiro_not_nil = shapiro(surgery_not_nil_group)

# 群間比較（マンホイットニーU検定）
stat, p_value = mannwhitneyu(surgery_nil_group, surgery_not_nil_group, alternative='two-sided')

# 箱ひげ図作成
plt.figure(figsize=(10, 6))
sns.boxplot(x='手術群', y='いつから（M）_数値', data=steroid_group, palette="pastel")
plt.title('手術の有無による「受診遅延」の比較', fontsize=14, weight='bold')
plt.ylabel('受診遅延（M）')
plt.xlabel('手術の有無')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# 統計結果表示
{
    "手術なし群（いつからM）": {"件数": len(surgery_nil_group), "中央値": surgery_nil_group.median(), "平均": surgery_nil_group.mean()},
    "手術あり群（いつからM）": {"件数": len(surgery_not_nil_group), "中央値": surgery_not_nil_group.median(), "平均": surgery_not_nil_group.mean()},
    "正規性検定（手術なし群）": {"統計量": shapiro_nil.statistic, "p値": shapiro_nil.pvalue},
    "正規性検定（手術あり群）": {"統計量": shapiro_not_nil.statistic, "p値": shapiro_not_nil.pvalue},
    "マンホイットニーU検定": {"統計量": stat, "p値": p_value}
}


In [ ]:
import pandas as pd
import scipy.stats as stats

# カテゴリ分類関数の定義
def categorize_m_new(value):
    if pd.isna(value) or str(value).strip() in ['未', '？']:
        return '未診断'
    try:
        value = float(value)
        if value <= 3:
            return '0-3ヶ月'
        elif value <= 6:
            return '4-6ヶ月'
        elif value <= 12:
            return '7-12ヶ月'
        elif value > 12:
            return '13ヶ月以上'
    except:
        return '不明'

# 必要な列を特定
steroid_col = [col for col in df.columns if 'ステロイド' in col][0]
thyroid_diagnosis_col = [col for col in df.columns if '甲状腺疾患診断済み' in col][0]
surgery_col = [col for col in df.columns if '手術' in col][0]

# ステロイド治療==1の群に絞り込み
steroid_group = df[df[steroid_col] == 1].copy()

# 診断期間カテゴリ列を作成
steroid_group['甲状腺疾患診断期間カテゴリ'] = steroid_group[thyroid_diagnosis_col].apply(categorize_m_new)

# クロス集計表の作成
contingency_table_steroid = pd.crosstab(
    steroid_group['甲状腺疾患診断期間カテゴリ'],
    steroid_group[surgery_col].apply(lambda x: '手術あり' if pd.notna(x) and str(x).strip() != '' else '手術なし')
)

# カイ二乗検定の実施
chi2_steroid, p_steroid, dof_steroid, expected_steroid = stats.chi2_contingency(contingency_table_steroid)

# 期待度数のデータフレーム作成
expected_df = pd.DataFrame(
    expected_steroid,
    index=contingency_table_steroid.index,
    columns=contingency_table_steroid.columns
)

# 結果表示
print("### クロス集計表（実測値）\n", contingency_table_steroid, "\n")
print("### カイ二乗検定結果")
print(f"- カイ二乗統計量: {chi2_steroid:.4f}")
print(f"- 自由度: {dof_steroid}")
print(f"- p値: {p_steroid:.4f}\n")
print("### 期待度数表（理論的に期待される値）\n", expected_df)


### クロス集計表（実測値）
 手術             手術あり  手術なし
甲状腺疾患診断期間カテゴリ            
0-3ヶ月             4    24
13ヶ月以上           10    23
4-6ヶ月             0     7
7-12ヶ月            3     5
未診断              10    23 

### カイ二乗検定結果
- カイ二乗統計量: 5.7364
- 自由度: 4
- p値: 0.2197

### 期待度数表（理論的に期待される値）
 手術                 手術あり       手術なし
甲状腺疾患診断期間カテゴリ                     
0-3ヶ月          6.935780  21.064220
13ヶ月以上         8.174312  24.825688
4-6ヶ月          1.733945   5.266055
7-12ヶ月         1.981651   6.018349
未診断            8.174312  24.825688


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 📊 データ作成（提供されたクロス集計表を使用）
data = {
    "甲状腺疾患診断期間カテゴリ": ["0-3ヶ月", "13ヶ月以上", "4-6ヶ月", "7-12ヶ月", "未診断"],
    "手術あり": [4, 10, 0, 3, 10],
    "手術なし": [24, 23, 7, 5, 23]
}

# DataFrameに変換
df_plot = pd.DataFrame(data)

# 各行を合計100%に変換（100%積み上げ棒グラフ用）
df_percentage = df_plot.set_index("甲状腺疾患診断期間カテゴリ")
df_percentage = df_percentage.div(df_percentage.sum(axis=1), axis=0) * 100

# カテゴリ順序を指定して並べ替え
category_order = ["0-3ヶ月", "4-6ヶ月", "7-12ヶ月", "13ヶ月以上", "未診断"]
df_percentage = df_percentage.loc[category_order]

# Seabornテーマ設定
sns.set_theme(style="whitegrid")
sns.set(font='IPAexGothic', font_scale=2.5)  # font_scale を2.5に増加


# 🗂️ 100%積み上げ棒グラフ作成
plt.figure(figsize=(10, 6))
df_percentage.plot(
    kind='bar',
    stacked=True,
    color=sns.color_palette("pastel"),
    edgecolor='black'
)

# グラフの装飾
plt.title("甲状腺疾患診断期間カテゴリ別 手術有無の割合（100%積み上げ棒グラフ）", fontsize=14, weight='bold')
plt.xlabel("甲状腺疾患診断期間カテゴリ", fontsize=12)
plt.ylabel("割合 (%)", fontsize=12)
plt.xticks(rotation=20, fontsize=10)
plt.yticks(fontsize=10)
plt.legend(title="手術有無", fontsize=10, title_fontsize=11, loc='upper right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# グラフ表示
plt.show()
